<a href="https://colab.research.google.com/github/emimorgan97/Machine-Learning-Final-Project/blob/main/NMF_full_image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import gzip
import h5py
import numpy as np
import shutil
import zipfile
from io import BytesIO

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
'''import numpy as np
import h5py
from sklearn.decomposition import NMF
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import gzip

# Function to read a batch of images from HDF5 file
def read_images(images_file_path, labels_file_path, class_index, batch_size):
    with gzip.open(images_file_path, 'rb') as f_images:
        with h5py.File(f_images, 'r') as hf_images:
            with gzip.open(labels_file_path, 'rb') as f_labels:
                with h5py.File(f_labels, 'r') as hf_labels:
                    images = hf_images['x']
                    labels = hf_labels['y']
                    class_indices = np.where(labels[:] == class_index)[0]
                    batch_indices = class_indices[:batch_size]
                    batch_data = images[batch_indices]
    return batch_data

# Load batch of 200 images of class 0
images_h5_gz_file_path = '/content/drive/MyDrive/ML_Data/pcamv1/camelyonpatch_level_2_split_train_x.h5.gz'
labels_h5_gz_file_path = '/content/drive/MyDrive/ML_Data/pcamv1/camelyonpatch_level_2_split_train_y.h5.gz'
class_index = 0
batch_size = 200
batch_data = read_images(images_h5_gz_file_path, labels_h5_gz_file_path, class_index, batch_size)

'''

In [ ]:
'''# Standardize the data
scaler = MinMaxScaler()
batch_data_scaled = scaler.fit_transform(batch_data.reshape(batch_size, -1))

# Initialize lists to store explained variance
explained_variance = []

# Define a range of number of components to try
n_components_range = range(20, 100)  # Adjust range as needed

# Iterate over the number of components
for n_components in n_components_range:
    # Create NMF model
    print(n_components)
    nmf = NMF(n_components=n_components, init='nndsvd', max_iter=250, tol=2e-2)

    # Fit NMF model to the data
    nmf.fit(batch_data_scaled)

    # Calculate the explained variance ratio manually
    reconstructed_data = np.dot(nmf.transform(batch_data_scaled), nmf.components_)
    explained_variance.append(1 - np.sum((batch_data_scaled - reconstructed_data) ** 2) / np.sum(batch_data_scaled ** 2))

# Plot the elbow curve
plt.plot(n_components_range, explained_variance, marker='o')
plt.xlabel('Number of Components')
plt.ylabel('Explained Variance')
plt.title('Elbow Curve for NMF')
plt.show()
'''


In [ ]:
'''#for class 1

# Load batch of 200 images of class 0
images_h5_gz_file_path = '/content/drive/MyDrive/ML_Data/pcamv1/camelyonpatch_level_2_split_train_x.h5.gz'
labels_h5_gz_file_path = '/content/drive/MyDrive/ML_Data/pcamv1/camelyonpatch_level_2_split_train_y.h5.gz'
class_index = 1
batch_size = 200
batch_data = read_images(images_h5_gz_file_path, labels_h5_gz_file_path, class_index, batch_size)

# Standardize the data
scaler = StandardScaler()
batch_data_scaled = scaler.fit_transform(batch_data.reshape(batch_size, -1))

# Initialize lists to store explained variance
explained_variance = []

# Define a range of number of components to try
n_components_range = range(1, 21)  # Adjust range as needed

# Iterate over the number of components
for n_components in n_components_range:
    # Create NMF model
    nmf = NMF(n_components=n_components, init='nndsvd', tol=2e-2)

    # Fit NMF model to the data
    nmf.fit(batch_data_scaled)

    # Calculate explained variance and append to the list
    explained_variance.append(nmf.explained_variance_ratio_.sum())

# Plot the elbow curve
plt.plot(n_components_range, explained_variance, marker='o')
plt.xlabel('Number of Components')
plt.ylabel('Explained Variance')
plt.title('Elbow Curve for NMF')
plt.show()
'''

In [ ]:
import numpy as np
import h5py
import gzip
import shutil
from sklearn.decomposition import NMF
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

# Function to apply NMF to the data
def apply_nmf_and_save(images_file_path, labels_file_path, output_file_path, batch_size, n_components, class_index, max_batches=None):
    nmf = NMF(n_components=n_components, init='random', tol=2e-2)
    with h5py.File(output_file_path, 'w') as hf_out:
        nmf_group = hf_out.create_group('nmf_data')

        with gzip.open(images_file_path, 'rb') as f:
            with h5py.File(f, 'r') as hf_images:
                with gzip.open(labels_file_path, 'rb') as f_labels:
                    with h5py.File(f_labels, 'r') as hf_labels:
                        images = hf_images['x']
                        labels = hf_labels['y']

                        class_indices = np.where(labels[:] == class_index)[0]
                        num_samples = len(class_indices)

                        # Calculate the number of batches to process
                        if max_batches:
                            num_batches = min(num_samples // batch_size, max_batches)
                        else:
                            num_batches = num_samples // batch_size

                        # Process each batch
                        for batch_num in range(num_batches):
                            print(batch_num)

                            start_idx = batch_num * batch_size
                            end_idx = min(start_idx + batch_size, num_samples)
                            batch_indices = class_indices[start_idx:end_idx]
                            batch_data = images[batch_indices]

                            # Flatten the batch data
                            batch_data_flattened = batch_data.reshape(batch_data.shape[0], -1)

                            # Apply NMF to the batch data
                            W = nmf.fit_transform(batch_data_flattened)
                            H = nmf.components_
                            nmf_data = np.dot(W, H)

                            # Concatenate original image data with NMF-transformed data
                            concatenated_data = np.concatenate((batch_data_flattened, nmf_data), axis=1)

                            # Store concatenated data in output .h5 file
                            nmf_group.create_dataset(f'batch_{start_idx}', data=concatenated_data)

                            # Stop if max_batches is reached
                            if max_batches and batch_num + 1 >= max_batches:
                                break


In [ ]:
# Example usage
images_h5_gz_file_path = '/content/drive/MyDrive/ML_Data/pcamv1/camelyonpatch_level_2_split_train_x.h5.gz'
labels_h5_gz_file_path = '/content/drive/MyDrive/ML_Data/pcamv1/camelyonpatch_level_2_split_train_y.h5.gz'
output_h5_file_path = 'output_data_class0_nmf.h5'
gzipped_file_path = 'output_data_class0_nmf.h5.gz'
batch_size = 200
n_components = 30  # Number of components for NMF
class_index = 0  # Specify the index of the class you want to extract (e.g., 0 for the first class)

max_batches = 150  # Limit to 30,000 images
apply_nmf_and_save(images_h5_gz_file_path, labels_h5_gz_file_path, output_h5_file_path, batch_size, n_components, class_index, max_batches=max_batches)

# Compress the .h5 file
with open(output_h5_file_path, 'rb') as f_in:
    with gzip.open(gzipped_file_path, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

# Move the gzipped file to Google Drive
destination_drive_path = '/content/drive/MyDrive/'
shutil.copy(gzipped_file_path, destination_drive_path)

In [ ]:
images_h5_gz_file_path = '/content/drive/MyDrive/ML_Data/pcamv1/camelyonpatch_level_2_split_train_x.h5.gz'
labels_h5_gz_file_path = '/content/drive/MyDrive/ML_Data/pcamv1/camelyonpatch_level_2_split_train_y.h5.gz'
output_h5_file_path = 'output_data_class1_nmf.h5'
gzipped_file_path = 'output_data_class1_nmf.h5.gz'
batch_size = 200
n_components = 30
class_index = 1  # Specify the index of the class you want to extract (e.g., 0 for the first class)

max_batches = 150 # Limit to 30,000 images
apply_nmf_and_save(images_h5_gz_file_path, labels_h5_gz_file_path, output_h5_file_path, batch_size, n_components, class_index, max_batches=max_batches)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149


In [ ]:
# Compress the .h5 file
with open(output_h5_file_path, 'rb') as f_in:
    with gzip.open(gzipped_file_path, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

In [ ]:
# Move the gzipped file to Google Drive
destination_drive_path = '/content/drive/MyDrive'
shutil.copy(gzipped_file_path, destination_drive_path)

'/content/drive/MyDrive/output_data_class1_nmf.h5.gz'